In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import itertools
import os

%matplotlib inline
from finrl import config
from finrl import config_tickers
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/home/henry/miniconda3/envs/finrl/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
import datetime

In [3]:
from data_processor.indicator import add_all_indicators, add_tech_indicator, tech_indicator_only


In [4]:
import data_processor.config

In [5]:
price_df = pd.read_parquet("./data/sz50_price.parquet")
fund_df = pd.read_parquet("./data/sz50_fundament.parquet")


In [6]:
df = add_all_indicators(price_df, fund_df)


In [7]:
available_tics = df.set_index('date').loc[data_processor.config.FIRST_DAY_OF_2009].tic.tolist()

In [8]:
tmp_list = []
for tic in available_tics:
    tmp_df = df[df['tic'] == tic]
    tmp_list.append(tmp_df)
    
df = pd.concat(tmp_list, ignore_index=True)
    

In [9]:
train = data_split(df, "2009-01-01", "2020-01-01")
trade = data_split(df, "2020-01-01", "2022-01-01")


In [10]:
indicators = data_processor.config.INDICATORS
# indicators = ["SMA_20", "SMA_60", "SMA_120", "macdhist", "CCI", "RSI", "NATR", "ADOSC"]


In [11]:
n_indicators = len(indicators)
n_indicators


17

In [12]:
stock_dimension = len(df.tic.unique())
stock_dimension


34

In [13]:
state_space = 1 + (2 + n_indicators) * stock_dimension
state_space


647

In [14]:
env_kwargs = {
    "hmax": 1000,
    "initial_amount" : 10000000,
    # "initial_list": [10000000] + [0 for i in range(stock_dimension)],
    # buy and sell cost for each stock
    "num_stock_shares" : [0] * stock_dimension,
    "buy_cost_pct": [0.001] * stock_dimension,
    "sell_cost_pct": [0.001] * stock_dimension,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": indicators,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
}

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [16]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


### SAC

In [17]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 256,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS, tensorboard_log=config.TENSORBOARD_LOG_DIR)

{'batch_size': 256, 'buffer_size': 100000, 'learning_rate': 5e-05, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device


In [18]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

Logging to tensorboard_log/sac_1
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 53       |
|    time_elapsed    | 200      |
|    total_timesteps | 10700    |
| train/             |          |
|    actor_loss      | 1.91e+06 |
|    critic_loss     | 1.32e+10 |
|    ent_coef        | 0.164    |
|    ent_coef_loss   | 632      |
|    learning_rate   | 5e-05    |
|    n_updates       | 10599    |
|    reward          | 3.448003 |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 53       |
|    time_elapsed    | 399      |
|    total_timesteps | 21400    |
| train/             |          |
|    actor_loss      | 1.39e+06 |
|    critic_loss     | 4.86e+08 |
|    ent_coef        | 0.28     |
|    ent_coef_loss   | 446      |
|    learning_rate   | 5e-05    |
|    n_updates       | 21299    |
|    reward    

### DDPG

In [ ]:
DDPG_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001
}

agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS)

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=10000)

## Backtest

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, **env_kwargs)

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

In [ ]:
df_account_value.tail()

In [ ]:
df_account_value.to_csv("./sz50_with_fund.csv")

In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

In [ ]:
from data_processor.ChinaStockDownloader import single_stock_query

baseline_df = single_stock_query("2020-01-01", "2022-01-01", "sh.000016")

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = single_stock_query("2020-01-01", "2022-01-01", "sh.000016")
baseline_df = baseline_df.apply(pd.to_numeric, errors='ignore')
stats = backtest_stats(baseline_df, value_col_name = 'close')

In [ ]:
test_ret = get_daily_return(df_account_value)
base_ret = get_daily_return(baseline_df, 'close')

In [ ]:
with_fund_value = pd.read_csv("./sz50_with_fund.csv")
with_fund_ret = get_daily_return(with_fund_value)

In [ ]:
print("==============Compare to SZ50===========")
%matplotlib inline

import pyfolio

In [ ]:
test_ret.index = pd.DatetimeIndex(test_ret.index.date)
base_ret.index = pd.DatetimeIndex(base_ret.index.date)

In [ ]:
with_fund_ret.index = pd.DatetimeIndex(with_fund_ret.index.date)

In [ ]:
test_ret

In [ ]:
with_fund_ret

In [ ]:
import quantstats as qs

qs.reports.html(with_fund_ret, test_ret)

In [ ]:
pyfolio.create_full_tear_sheet(returns=test_ret, benchmark_rets=base_ret)